`parse_results.ipynb`: parse results from database into CSV.

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
# imports
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
from auctions.webutils import *

from open_spiel.python.examples.ubc_cma import get_results, get_game_info, get_algorithm_from_run, analyze_samples

# TODO: run this analysis in the jobs
from open_spiel.python.algorithms.exploitability import nash_conv
from open_spiel.python.examples.ubc_decorators import ModalAgentDecorator

In [3]:
experiments = [
    'feb3_test_short_v4'
]

csv_path = 'results/feb3_test_short_v4.csv'

In [4]:
runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")

Found 100 runs


In [9]:
records = []
for run in tqdm(runs):    
    game = run.game.load_as_spiel()
    record = {
        'run_name': run.name,
        'game_name': run.game.name, 
        'seed': run.config.get('seed'), 
        'config': run.get_config_name(),
        'alg': get_algorithm_from_run(run),
    }
    
    record.update(get_game_info(game, run.game))  
    
    record['no_error'] = False
    records.append(record) # Put it here so you see the False's in the display
        
    try:
        game, final_checkpoint, _ = get_results(run, load_policy=False)
    except Exception as e:
        print(f"Skipping run {run.name} because of error {e}")
        continue
    
    try:
        record['t'] = final_checkpoint.t
        record['walltime'] = run.walltime()

        for (evaluation, prefix) in [
            (final_checkpoint.get_modal_eval(), ''),
            (final_checkpoint.get_straightforward_eval(), 'straightforward_')
        ]:
            record[f'{prefix}nash_conv'] = evaluation.nash_conv
            record[f'{prefix}rewards'] = evaluation.mean_rewards
            record[f'{prefix}nash_conv_frac'] = evaluation.nash_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.nash_conv) else np.nan
            record[f'{prefix}heuristic_conv'] = evaluation.heuristic_conv
            record[f'{prefix}heuristic_conv_frac'] = evaluation.heuristic_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.heuristic_conv) else np.nan
    
            for i in range(game.num_players()):
                record[f'{prefix}rewards_{i}'] = evaluation.mean_rewards[i]
                record[f'{prefix}nc_player_improvements_{i}'] = evaluation.nash_conv_player_improvements[i] if not pd.isnull(evaluation.nash_conv) else np.nan
                record[f'{prefix}nc_player_improvements_frac_{i}'] = (evaluation.nash_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.nash_conv) else np.nan
    
                record[f'{prefix}hc_player_improvements_{i}'] = evaluation.heuristic_conv_player_improvements[i] if not pd.isnull(evaluation.heuristic_conv) else np.nan
                record[f'{prefix}hc_player_improvements_frac_{i}'] = (evaluation.heuristic_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.heuristic_conv) else np.nan

            sample_metrics = analyze_samples(evaluation.samples, game)
            sample_metrics = {f'{prefix}{k}': sample_metrics[k] for k in sample_metrics} 
            record.update(**sample_metrics)
        record['no_error'] = True
    except Exception as e:
        print(f"Something wrong with {run}. Skipping. {e}")
        # raise e
        # break
        import traceback
        print(traceback.format_exc())

print(f'Parsed results for {len(records)} runs.')

100%|██████████| 100/100 [01:55<00:00,  1.16s/it]

Parsed results for 100 runs.


In [10]:
df = pd.DataFrame.from_records(records)
df.to_csv(csv_path, index=False)